In [2]:
import boto3
import hashlib
import string
import os
import json
root_dir = '../Themes'
audio_dir = '../audio'

In [ ]:


print(os.listdir(root_dir))

for subdir, _, files in os.walk(root_dir):
    for file in files:
        if file.endswith('.txt'):
            output = []
            unique_no_sub = set()
            unique_sub = set()
            theme = os.path.splitext(file)[0]
            file_path = os.path.join(subdir, file)
            try:
                #print(f"Processing {file_path}...")
                with open(file_path, 'r', encoding='utf-8') as f:
                    for line_number, line in enumerate(f, 1):
                        line = line.strip()
                        if not line or line.startswith('Description:'):
                            continue
                        parts = line.split('\t')
                        if len(parts) != 4:
                            print(f"Warning: Line {line_number} in {file} does not have 4 columns.")
                            continue
                        difficulty, line_no_sub, line_with_sub, dom_flag = parts
                        dominant = "Master" if dom_flag.lower() == "dom" else None
                        # Entry without submissive pet name
                        if line_no_sub in unique_no_sub:
                            print(f"Warning: Duplicate entry found in {file}: {line_no_sub}")
                        else:
                            entry_no_sub = {
                                "type": "audio",
                                "line": line_no_sub,
                                "theme": theme,
                                "dominant": dominant,
                                "submissive": None,
                                "difficulty": difficulty
                            }
                            output.append(entry_no_sub)
                            unique_no_sub.add(line_no_sub)
                        if line_with_sub in unique_sub:
                            print(f"Warning: Duplicate entry found in {file}: {line_with_sub}")
                        else:
                            # Entry with submissive pet name
                            entry_with_sub = {
                                "type": "audio",
                                "line": line_with_sub,
                                "theme": theme,
                                "dominant": dominant,
                                "submissive": "Bambi",
                                "difficulty": difficulty
                            }
                            output.append(entry_with_sub)
                            unique_sub.add(line_with_sub)
            except Exception as e:
                print(f"Error processing file {file}: {e}")
            if output:
                with open(os.path.join(subdir,f'{theme}.json'), 'w', encoding='utf-8') as outfile:
                    json.dump(output, outfile, ensure_ascii=False, indent=4)
            else:
                print(f"No valid entries found in {file}")

In [ ]:
polly = boto3.client('polly')

os.makedirs(audio_dir, exist_ok=True)

processed_lines = set()
# Include existing mp3 files in processed_lines
for filename in os.listdir(audio_dir):
    if filename.endswith('.mp3'):
        file_root = os.path.splitext(filename)[0]  # Remove file extension
        processed_lines.add(file_root)

for subdir, _, files in os.walk(root_dir):
    for file in files:
        if file.endswith('.json'):
            new_entries = 0
            file_path = os.path.join(subdir, file)
            theme = os.path.splitext(file)[0]
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    for entry in data:
                        line = entry.get('line', '')
                        # Normalize the line to create a unique key
                        line_key = line.translate(str.maketrans('', '', string.punctuation)).replace(' ', '').lower()
                        line_hash = hashlib.sha256(line_key.encode('utf-8')).hexdigest()
                        if line_hash in processed_lines:
                            continue
                        response = polly.synthesize_speech(
                            Text=line,
                            OutputFormat='mp3',
                            VoiceId='Salli'  # Standard voice, not neural
                        )
                        new_entries += 1
                        audio_file_path = os.path.join(audio_dir, f"{line_hash}.mp3")
                        with open(audio_file_path, 'wb') as audio_file:
                            audio_file.write(response['AudioStream'].read())
                        processed_lines.add(line_hash)
                if new_entries:
                    print(f"Processed: {theme} with {new_entries} new entries")
            except Exception as e:
                print(f"Error processing file {file}: {e}")

In [ ]:
orphaned_lines = []

for subdir, _, files in os.walk(root_dir):
    for file in files:
        if file.endswith('.json'):
            file_path = os.path.join(subdir, file)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    for entry in data:
                        line = entry.get('line', '')
                        if not line:
                            orphaned_lines.append({'file': file, 'entry': entry, 'reason': 'Missing line'})
                            continue
                        # Generate line hash as before
                        line_key = line.translate(str.maketrans('', '', string.punctuation)).replace(' ', '').lower()
                        line_hash = hashlib.sha256(line_key.encode('utf-8')).hexdigest()
                        if line_hash not in processed_lines:
                            orphaned_lines.append({'file': file, 'line': line, 'reason': 'No audio associated'})
            except Exception as e:
                print(f"Error processing file {file}: {e}")

if orphaned_lines:
    print("The following entries are orphaned:")
    for orphan in orphaned_lines:
        print(f"File: {orphan['file']}, Line: {orphan.get('line', '')}, Reason: {orphan['reason']}")
else:
    print("No orphaned or abandoned lines found.")